In [151]:
import os
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import sys
import threading

In [2]:
tsp_dir = os.getcwd() + os.sep + 'TSPLIB' + os.sep

In [3]:
tsp_files_list = os.listdir(tsp_dir)

In [4]:
subprocess.run('make christofides', shell=True, capture_output=True)

CompletedProcess(args='make christofides', returncode=0, stdout=b"make: 'christofides' is up to date.\n", stderr=b'')

In [5]:
out = subprocess.run(['./christofides -f TSPLIB/gr17.tsp'], shell=True, capture_output=True)
output = out.stdout.decode('utf-8')
print(output)

OptimalSoln:	2085.000000
Simple:		1.053717	2197
Tri-Opt:	1.031655	2151
Tri-Comp:	1.031655	2151
Comp-Heur:	1.074820	2241



In [30]:
def parseOutputString(outputStr):
    lines = outputStr.split('\n')
    resultDict = {}
    for i in range(len(lines)-1):
        line = lines[i]
        s = line.split()
        if(i == 0):
            resultDict[i] = [s[1]]
        else:
            resultDict[i] = s[1:]
    return resultDict

In [31]:
parseOutputString(output)

{0: ['2085.000000'],
 1: ['1.053717', '2197'],
 2: ['1.031655', '2151'],
 3: ['1.031655', '2151'],
 4: ['1.074820', '2241']}

In [8]:
def getSupportedFilesList():
    tsp_files_list = os.listdir(tsp_dir)
    supportedFiles = []
    keyWord = 'EDGE_WEIGHT_SECTION'
    for filename in tsp_files_list:
        foundKeyWord = False
        file = open(tsp_dir + os.sep + filename, 'r')
        lines = file.readlines()
        for line in lines:
            if(line.find(keyWord) != -1):
                foundKeyWord = True
                break
        if(foundKeyWord == False):
            supportedFiles.append(filename)
    return supportedFiles

In [9]:
supported_tsp_files_list = getSupportedFilesList()

In [164]:
def launchChristofides(filename = 'gr48', perturbation_percentage = 0.0):
    filename = filename + '.tsp'
    if(filename not in supported_tsp_files_list):
        print('WARNING: You are running the algorithm on an unsupported file, Perturbation does not work.')
    filename = 'TSPLIB' + os.sep + filename
    command = './christofides' + ' -f ' + filename + ' -p ' + str(perturbation_percentage)
    print(command)
    out = subprocess.run([command], shell=True, capture_output=True)
    output = out.stdout.decode('utf-8')
    resultDict = parseOutputString(output)
#     print(resultDict)
    return resultDict

In [165]:
launchChristofides('gr431', 0.01)

./christofides -f TSPLIB/gr431.tsp -p 0.01


{0: ['171414.000000'],
 1: ['1.123957', '192662'],
 2: ['1.068728', '183195'],
 3: ['1.050591', '180086'],
 4: ['1.088692', '186617']}

In [166]:
MAX_THREADS = 1

threads = []
for i in range(MAX_THREADS):
    t = threading.Thread(target = launchChristofides, args = ('gr431', 0.01))
    threads.append(t)
    t.start()
for t in threads:
    t.join()

./christofides -f TSPLIB/gr431.tsp -p 0.01


In [167]:
def launchChristofidesThreadSafe(resultDictList, listLock, filename = 'gr48', perturbation_percentage = 0.0):
    dictToAdd = launchChristofides(filename, perturbation_percentage)
    listLock.acquire()
    resultDictList.append(dictToAdd)
    listLock.release()
    return    

In [169]:
resultList = []
listLock = threading.Lock()
threads = []
for i in range(10):
    t = threading.Thread(target = launchChristofidesThreadSafe, args = (resultList, listLock, 'gr431', 0.01))
    threads.append(t)
    t.start()
for t in threads:
    t.join()

print(len(resultList))
print(resultList)

./christofides -f TSPLIB/gr431.tsp -p 0.01
./christofides -f TSPLIB/gr431.tsp -p 0.01
./christofides -f TSPLIB/gr431.tsp -p 0.01
./christofides -f TSPLIB/gr431.tsp -p 0.01
./christofides -f TSPLIB/gr431.tsp -p 0.01
./christofides -f TSPLIB/gr431.tsp -p 0.01
./christofides -f TSPLIB/gr431.tsp -p 0.01
./christofides -f TSPLIB/gr431.tsp -p 0.01
./christofides -f TSPLIB/gr431.tsp -p 0.01./christofides -f TSPLIB/gr431.tsp -p 0.01

10
[{0: ['171414.000000'], 1: ['1.116245', '191340'], 2: ['1.067766', '183030'], 3: ['1.050194', '180018'], 4: ['1.087752', '186456']}, {0: ['171414.000000'], 1: ['1.125853', '192987'], 2: ['1.070035', '183419'], 3: ['1.050066', '179996'], 4: ['1.081977', '185466']}, {0: ['171414.000000'], 1: ['1.126092', '193028'], 2: ['1.069994', '183412'], 3: ['1.050142', '180009'], 4: ['1.081714', '185421']}, {0: ['171414.000000'], 1: ['1.122050', '192335'], 2: ['1.067538', '182991'], 3: ['1.049588', '179914'], 4: ['1.088091', '186514']}, {0: ['171414.000000'], 1: ['1.125976',